In [ ]:
from google.colab import drive
from pandas import DataFrame
import pandas as pd
import os

drive.mount('/content/drive')

folder = "drive/Othercomputers/Il mio Laptop/Universita/[IA] Artificial Intelligence/[HLT] Human Language Technologies/project/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
datasets = []

# We chose to avoid the lemmatized version of word, because the berts model work better with the raw version of word
datasets.append(pd.read_csv(folder+"Corpus/anamnesi.a.iob",sep="\t",
                  names=["word","entity"],usecols=[0,3]).astype({'word': 'string','entity': 'string'}))

datasets.append(pd.read_csv(folder+"Corpus/anamnesi.b.iob",sep="\t",
                  names=["word","entity"],usecols=[0,3]).astype({'word': 'string','entity': 'string'}))

datasets.append(pd.read_csv(folder+"Corpus/esami.a.iob",sep="\t",
                  names=["word","entity"],usecols=[0,3]).astype({'word': 'string','entity': 'string'}))

datasets.append(pd.read_csv(folder+"Corpus/esami.b.iob",sep="\t",
                  names=["word","entity"],usecols=[0,3]).astype({'word': 'string','entity': 'string'}))


In [ ]:
def buildDataset(datasets : list, file_name : str) -> None:

  # ===== Parameters =====
  separator = "%:%" # Separate the text and label in the temporary dataset
  tmp_file = "tmp.csv" # name of temporary file
  stop_char = [".",";",":"] # list of character used to create a phrase
  char_per_phrase = 300 # Max number of words per phrase
  # ===== Parameters =====

  try:
    os.remove(file_name) # Remove old dataset
  except:
    None
  
  f = open(tmp_file, "a") # Store a temporary result on file
  nds = 0 # Count of number of dataset

  for df in datasets:

    print("Dataset: ",nds)
    string,labels = "",""
    nphrases = 0 # Number of phrased captured
    nwords = 0 # Number of word in a phrase
    flag = False # True if it's found a significant entity tag

    # ===== Remove the meaningless values =====
    first = len(df) # Number of rows before the elimination
    df.dropna(how='any',axis=0,inplace=True)
    df.drop(df[df.word == "-DOCSTART-"].index,inplace=True)
    print("\tRow deleted from raw-dataset: ",first-len(df))
    print("\tNum of word: " ,len(df))
    # ===== Remove the meaningless values =====

    # ===== Build and Concat the phrases =====
    # Iterate all row in the dataframe (each row corresponding to a word and label)
    for _,row in df.iterrows():
      # We concatenate each word with the actual string, the same things with label
      string += " " + row[0]
      labels  += " " + row[1]
      nwords += 1

      if row[1] != "O": # True if it's found a significant entity tag
        flag = True

      # If we encouter these chars or the number of word exceed 512
      # means that the phrase is finished so we write the 
      # phrase on the file and we clear the string and label buffers
      if (row[0] in stop_char) | (nwords >= char_per_phrase):

        # If the flag is true means that inside the phrase there is at least
        # a signigificant entity, so we take it into consideration, otherwise,
        # we do not store it. (Reduce the dimentions of database)
        if flag : 
          f.write(string + separator + labels + "\n")
          nphrases += 1
        string,labels = "",""
        nwords = 0
        flag = False

    print("\tNumber of phrases :",nphrases)
    print("")
    nds += 1
    # ===== Build and Concat the phrases =====

  f.close()

  # ===== Remove the duplicate  =====
  # After we have finished to write the file we load it 
  # in order to remove the duplicate rows 
  dataset = pd.read_csv(tmp_file,sep=separator,
                        names=["text","labels"],engine="python")
  
  os.remove(tmp_file) # Removing the temporary file

  first = len(dataset) # Number of rows before the elimination
  print("Number of phrased achieved: ",first)
  dataset.drop_duplicates(inplace=True)
  print("Number of duplicated deleted: ",first - len(dataset))
  # ===== Remove the duplicate  =====

  # ===== Exporting the dataframe  =====
  dataset.to_csv(file_name, index=False)
  # ===== Exporting the dataframe  =====

  return

In [ ]:
buildDataset(datasets,"dataset.csv")

! cp -r "/content/dataset.csv" "drive/Othercomputers/Il mio Laptop/Universita/[IA] Artificial Intelligence/[HLT] Human Language Technologies/project/Sources/dataset.csv"

Dataset:  0
	Row deleted from raw-dataset:  10586
	Num of word:  1312283
	Number of phrases : 48456

Dataset:  1
	Row deleted from raw-dataset:  10586
	Num of word:  1312283
	Number of phrases : 20525

Dataset:  2
	Row deleted from raw-dataset:  26771
	Num of word:  2307161
	Number of phrases : 56235

Dataset:  3
	Row deleted from raw-dataset:  26771
	Num of word:  2307161
	Number of phrases : 43133

Number of phrased achieved:  175658
Number of duplicated deleted:  105728


In [ ]:
# Dataset:  0
# 	Row deleted from raw-dataset:  10586
# 	Num of word:  1312283
# 	Number of phrases : 77974

# Dataset:  1
# 	Row deleted from raw-dataset:  10586
# 	Num of word:  1312283
# 	Number of phrases : 77974

# Dataset:  2
# 	Row deleted from raw-dataset:  26771
# 	Num of word:  2307161
# 	Number of phrases : 236166

# Dataset:  3
# 	Row deleted from raw-dataset:  26771
# 	Num of word:  2307161
# 	Number of phrases : 236166

# Number of phrased achieved:  628280
# Number of duplicated deleted:  458891